In [ ]:
# -*- coding: utf-8 -*-
# Mini Neural Network for classifying even and odd numbers
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.datasets import mnist
from keras.models import load_model

import time
import cv2
import numpy as np
import gc
from HW4.logger import Logger
import matplotlib.pyplot as plt


def plot_curve(history, epochs):
    # # summarize history for accuracy
    # plt.plot(history.history['acc'])
    # plt.plot(history.history['val_acc'])
    # plt.title('model accuracy')
    # plt.ylabel('accuracy')
    # plt.xlabel('epoch')
    # plt.legend(['train', 'test'], loc='upper left')
    # plt.show()
    #
    # # summarize history for loss plt.plot(history.history['loss'])
    # # plt.plot(history.history['val_loss']) plt.title('model loss')
    # plt.ylabel('loss')
    # plt.xlabel('epoch')
    # plt.legend(['train', 'test'], loc='upper left')
    # plt.show()

    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0, epochs), history.history["loss"], label="train_loss")
    plt.plot(np.arange(0, epochs), history.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, epochs), history.history["acc"], label="train_acc")
    plt.plot(np.arange(0, epochs), history.history["val_acc"], label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="upper right")
    plt.show()
    # plt.savefig("Loss_Accuracy_alexnet_{:d}e.jpg".format(epochs))


# Load image from file
def load_image(file):
    img = cv2.imread(file)
    cv2.imshow('Original Image', img)
    cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return img


# Resize image to 28*28
def resize_image(img):
    res = cv2.resize(img, (28, 28), interpolation=cv2.INTER_CUBIC)
    image_bw = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    cv2.imshow('Resize and grayscale Image', image_bw)
    cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return img


def grayscale(img):
    image_bw = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imshow('Grayscale Image', image_bw)
    cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # return img
    return image_bw


def binary_image(img):
    # img = cv2.GaussianBlur(img, (5, 5), 0)
    img = cv2.medianBlur(img, 5)
    th3 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    cv2.imshow('Binary image', th3)
    cv2.waitKey(0)
    return th3


def process_image(img):
    # img = cv2.imread("/Users/junruigong/Desktop/image.png")
    img = cv2.resize(img, (28, 28), interpolation=cv2.INTER_CUBIC)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    cv2.imshow('Grayscale Image', img)
    cv2.waitKey(0)
    # img = cv2.GaussianBlur(img, (5, 5), 0)
    cv2.imshow('GaussianBlur Image', img)
    cv2.waitKey(0)
    th3 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    cv2.imshow('Binary Image', th3)
    cv2.waitKey(0)
    return th3


def relabel(labels):
    for idx, item in enumerate(labels):
        if item % 2 == 0:
            labels[idx] = 0  # even number
        else:
            labels[idx] = 1  # odd number
    return labels


def get_data():
    (train_picture, train_label), (test_picture, test_label) = mnist.load_data()

    # data = input_data.read_data_sets('data/MNIST', one_hot=True)

    # print("- Training-set:\t\t{}".format(data.num_train))
    train_picture = train_picture.reshape(60000, 28, 28, 1).astype('float32')
    test_picture = test_picture.reshape(10000, 28, 28, 1).astype('float32')

    train_picture = train_picture / 255
    test_picture = test_picture / 255

    # Reset label as odd or even
    train_label = relabel(train_label)
    test_label = relabel(test_label)

    train_label = to_categorical(train_label, 2)
    test_label = to_categorical(test_label, 2)
    return train_picture, train_label, test_picture, test_label


def reshape_image(img):
    img = img.reshape(1, 28, 28, 1).astype('float32')
    img = img / 255

    return img


def log_result(model, test_picture, test_label):
    classes = ['even number', 'odd number']
    predicted_classes = []
    ground_truth = []
    predictions = model.predict(test_picture)
    for item in test_label:
        # print(np.argmax(item))
        ground_truth.append(np.argmax(item))
    for item in predictions:
        # print(np.argmax(item))
        predicted_classes.append(np.argmax(item))
    print(model.summary())
    my_logger = Logger()
    my_logger.log_confusion(ground_truth, predicted_classes, classes)


def train_model():
    train_picture, train_label, test_picture, test_label = get_data()
    # input_shape = (28, 28,1)

    # Define model
    model = Sequential()

    # 2 Convolution layer
    model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)))
    model.add(Conv2D(64, kernel_size=(5, 5), activation='relu'))

    # Pooling layer
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Dropout rate of 40%
    model.add(Dropout(0.4, noise_shape=None, seed=None))

    model.add(Flatten())

    model.add(Dense(128, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    # Compile network
    # Use gradient descent optimization and a learning rate of 0.001
    sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    # Train model
    # Train for 5 epochs
    start_time = time.time()
    history = model.fit(train_picture, train_label, batch_size=64, epochs=5, validation_data=(test_picture, test_label))

    plot_curve(history, 5)

    # Evaluate model
    loss, accuracy = model.evaluate(test_picture, test_label, verbose=0)

    print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy * 100))

    print("\n---------------------------------------------")
    print("\nRuntime: " + str(int((time.time() - start_time) / 60)) + " minutes")
    print("\n---------------------------------------------")

    # Save result
    log_result(model, test_picture, test_label)

    gc.collect()

    return model


def main():

    # Training model
    # model = train_model()
    #
    # print('Train completed')
    # model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
    # del model  # deletes the existing model

    # Load the pre-trained model
    model = load_model('my_model.h5')

    # train_picture, train_label, test_picture, test_label = get_data()
    # log_result(model, test_picture, test_label)

    print("\n---------------------------------------------------------------------"
          "\nAfter seeing the image, please press any keys to continue the process"
          "\n---------------------------------------------------------------------\n")

    while 1:
        i = input('number:')
        j = input('type:')

        # Change file path firstly
        org_img = load_image('/Users/junruigong/Desktop/test_image/' + str(i) + '_' + str(j) + '.png')

        # Load image
        img = process_image(org_img)

        # Reshape image
        img = reshape_image(img)

        # 数据预测
        predicted_classes = []
        predictions = model.predict(img)
        print 'test[' + str(i) + '_' + str(j) + '], num=' + str(i) + ':'
        print(predictions[0])
        if np.argmax(predictions[0]) == 0:
            print('Even number')
        else:
            print('Odd number')

        cv2.destroyAllWindows()


if __name__ == "__main__":
    main()
